# American Time Use Survey Cleaning and Preprocessing

## - Sheldon Sebastian

This notebook is primarily for *cleaning and preprocessing* the American Time Use Survey (ATUS) Files. Due to limitations of github storage, the source data needs to be dowloaded from specified kaggle website and placed in folder named "data" in same directory of this Jupyter Notebook. 

### Source of Data:

This data is downloaded from the following website: 

https://www.kaggle.com/bls/american-time-use-survey

In [2]:
import pandas as pd
import numpy as np

**Respondent file:**

The Respondent file contains information about ATUS respondents, including their labor force status and earnings.

In [3]:
respondentData = pd.read_csv(
    "data/atusresp.csv",
    usecols=["tucaseid", "telfs", "trmjind1", "tufnwgtp",],
)

The first few records of respondent data looks like this:

In [4]:
respondentData.head()

,tucaseid,trmjind1,tufnwgtp,telfs
0,20030100013280,10,8155463.0,2
1,20030100013344,4,1735323.0,1
2,20030100013352,10,3830528.0,2
3,20030100013848,-1,6622023.0,4
4,20030100014165,10,3068387.0,1


In [5]:
# unique id for esach respondent
respondentData.rename({"tucaseid": "unique id"}, axis=1, inplace=True)

# telfs is employment status of respondent
# 1 Employed - at work
# 2 Employed - absent
# 3 Unemployed - on layoff
# 4 Unemployed - looking
# 5 Not in labor force
respondentData.rename({"telfs": "Respondent Labour Status"}, axis=1, inplace=True)

respondentData["Respondent Labour Status"] = respondentData[
    "Respondent Labour Status"
].map({1: "Employed", 2: "Employed", 3: "Unemployed", 4: "Unemployed", 5: np.NaN})

# trmjind1 is major industry code
respondentData.rename({"trmjind1": "Generalized industry code"}, axis=1, inplace=True)

respondentData["Generalized industry code"] = respondentData[
    "Generalized industry code"
].map(
    {
        1: "Agriculture, forestry, fishing, and hunting",
        2: "Mining",
        3: "Construction",
        4: "Manufacturing",
        5: "Wholesale and retail trade",
        6: "Transportation and utilities",
        7: "Information",
        8: "Financial activities",
        9: "Professional and business services",
        10: "Educational and health services",
        11: "Leisure and hospitality",
        12: "Other services",
        13: "Public administration",
    }
)

respondentData.head()

,unique id,Generalized industry code,tufnwgtp,Respondent Labour Status
0,20030100013280,Educational and health services,8155463.0,Employed
1,20030100013344,Manufacturing,1735323.0,Employed
2,20030100013352,Educational and health services,3830528.0,Employed
3,20030100013848,NaN,6622023.0,Unemployed
4,20030100014165,Educational and health services,3068387.0,Employed


**ATUS Weightage:**

ATUS is based on a stratified random sample, whereby some demographic groups are oversampled to ensure adequate sample size for detailed estimates. The weights ensure that each group is correctly represented in the population. These weights have *already* been calculated and are found on the Respondent file, under the "tufnwgtp" column.

In [6]:
# tufnwgtp = ATUS weights
respondentData.rename({"tufnwgtp": "Weightage"}, axis=1, inplace=True)

Replace all -1 (missing values) with NaN

In [7]:
respondentData = respondentData.replace(-1, np.NaN)

After processing the respondentData looks like this:

In [8]:
respondentData.head()

,unique id,Generalized industry code,Weightage,Respondent Labour Status
0,20030100013280,Educational and health services,8155463.0,Employed
1,20030100013344,Manufacturing,1735323.0,Employed
2,20030100013352,Educational and health services,3830528.0,Employed
3,20030100013848,NaN,6622023.0,Unemployed
4,20030100014165,Educational and health services,3068387.0,Employed


-------------------
**Roster file:**

The Roster file contains information about household members and nonhousehold children (under 18) of ATUS respondents. It includes information such as age and sex.

In [9]:
rosterData = pd.read_csv("data/atusrost.csv", usecols=["tucaseid", "terrp", "teage"])

The first few records of rosterData looks like this:

In [10]:
rosterData.head()

,tucaseid,terrp,teage
0,20030100013280,18,60
1,20030100013280,20,72
2,20030100013280,22,37
3,20030100013344,18,41
4,20030100013344,20,42


In [11]:
# rename for ease of use
rosterData.rename({"tucaseid": "unique id", "teage": "Age"}, axis=1, inplace=True)

# terrp =  how is this person related to you?
rosterData.rename({"terrp": "Activity Partner"}, axis=1, inplace=True)

# replace numbers with actual values
rosterData["Activity Partner"] = rosterData["Activity Partner"].map(
    {
        18: "Self",
        19: "Self",
        20: "Spouse",
        21: "Unmarried partner",
        22: "Own household child",
        23: "Grandchild",
        24: "Parent",
        25: "Brother/sister",
        26: "Other relative",
        27: "Foster child",
        28: "Housemate/roommate",
        29: "Roomer/boarder",
        30: "Other nonrelative",
        40: "Own nonhousehold child < 18",
    }
)

After processing the rosterData looks like this:

In [12]:
rosterData.head()

,unique id,Activity Partner,Age
0,20030100013280,Self,60
1,20030100013280,Spouse,72
2,20030100013280,Own household child,37
3,20030100013344,Self,41
4,20030100013344,Spouse,42


----------------

**Codes:**

The codes file contains all the mapping of code to the action performed. 

The ATUS activity lexicon *dictionary* can be found at: https://www.bls.gov/tus/lexiconnoex0315.pdf

In [13]:
activityCodes = pd.read_csv("data/codes.csv").set_index("code")

The first few rows of activityCodes looks like this:

In [14]:
activityCodes.head()

,name
code,
1,Personal Care Activities
101,Sleeping
102,Grooming
103,Health-related self care
104,Personal Activities


We create a dictionary to easily map the code to activity in our ATUS dataframe.

In [15]:
# create a dictionary using
activityDictionary = activityCodes.to_dict()["name"]

-----------------
**Activity file:**

The Activity file contains information about how ATUS respondents spent their diary day. It includes information such as activity codes, activity start and stop times, and locations.

In [16]:
activityData = pd.read_csv(
    "data/atusact.csv",
    usecols=["tucaseid", "tuactivity_n", "tuactdur24", "trtier1p", "tewhere"],
)

First few rows of activityData looks like this:

In [17]:
activityData.head()

,tucaseid,tuactivity_n,tuactdur24,trtier1p,tewhere
0,20030100013280,1,60,13,9
1,20030100013280,2,30,1,-1
2,20030100013280,3,600,1,-1
3,20030100013280,4,150,12,1
4,20030100013280,5,5,11,1


In [18]:
# primary key
activityData.rename({"tucaseid": "unique id"}, axis=1, inplace=True)

# tuactivity_n = activity serial number for a particular respondent
activityData.rename({"tuactivity_n": "activity serial number"}, axis=1, inplace=True)

# tuactdur24 = Duration of activity in minutes
activityData.rename({"tuactdur24": "minutes of activity"}, axis=1, inplace=True)

# trtier1p = first 2 digits of activity
activityData.rename(
    {"trtier1p": "Activity Category"}, axis=1, inplace=True,
)

# convert the activity numeric code into string using the activityDictionary
activityData["Activity Category"] = activityData["Activity Category"].map(
    activityDictionary
)

# tewhere = where were you during the activity?
activityData.rename({"tewhere": "Location of activity"}, axis=1, inplace=True)

# convert location code to location values
activityData["Location of activity"] = activityData["Location of activity"].map(
    {
        1: "Respondent's home or yard",
        2: "Respondent's workplace",
        3: "Someone else's home",
        4: "Restaurant or bar",
        5: "Place of worship",
        6: "Grocery store",
        7: "Other store/mall",
        8: "School",
        9: "Outdoors away from home",
        10: "Library",
        11: "Other place",
        12: "Car, truck, or motorcycle (driver)",
        13: "Car, truck, or motorcycle (passenger)",
        14: "Walking",
        15: "Bus",
        16: "Subway/train",
        17: "Bicycle",
        18: "Boat/ferry",
        19: "Taxi/limousine service",
        20: "Airplane",
        21: "Other mode of transportation",
        30: "Bank",
        31: "Gym/health club",
        32: "Post Office",
        89: "Unspecified place",
        99: "Unspecified mode of transportation",
    }
)

First few rows of processed activityData looks like this:

In [24]:
activityData.head()

,unique id,activity serial number,minutes of activity,Activity Category,Location of activity
0,20030100013280,1,60,"Sports, Exercise, & Recreation",Outdoors away from home
1,20030100013280,2,30,Personal Care Activities,NaN
2,20030100013280,3,600,Personal Care Activities,NaN
3,20030100013280,4,150,"Socializing, Relaxing, and Leisure",Respondent's home or yard
4,20030100013280,5,5,Eating and Drinking,Respondent's home or yard


-----------------
**CPS 2003-2015 file:**

The ATUS-CPS file contains information about each household member of all individuals selected to participate in ATUS. The information on the ATUS-CPS file was collected 2 to 5 months before the ATUS interview.

In [25]:
currentPopulationSurvey = pd.read_csv(
    "data/atuscps.csv",
    usecols=["tucaseid", "hufaminc", "pemaritl", "pesex"],
    low_memory=False,
)

# tucaseid or primary key
currentPopulationSurvey.rename({"tucaseid": "unique id"}, inplace=True, axis=1)

# hufaminc = Family income
# The combined income of all family members during the last 12 months
currentPopulationSurvey.rename(
    {"hufaminc": "Yearly Family income category (in dollars)"}, inplace=True, axis=1
)
# convert the family income code into categorical values
currentPopulationSurvey[
    "Yearly Family income category (in dollars)"
] = currentPopulationSurvey["Yearly Family income category (in dollars)"].map(
    {
        1: "Less than 10,000",
        2: "Less than 10,000",
        3: "Less than 10,000",
        4: "10,000 to 24,999",
        5: "10,000 to 24,999",
        6: "10,000 to 24,999",
        7: "10,000 to 24,999",
        8: "25,000 to 49,999",
        9: "25,000 to 49,999",
        10: "25,000 to 49,999",
        11: "25,000 to 49,999",
        12: "50,000 to 99,999",
        13: "50,000 to 99,999",
        14: "50,000 to 99,999",
        15: "100,000 to 149,999",
        16: "150,000 and over",
    }
)

# pemaritl= marriage status
currentPopulationSurvey.rename({"pemaritl": "Marriage Status"}, inplace=True, axis=1)
currentPopulationSurvey["Marriage Status"] = currentPopulationSurvey[
    "Marriage Status"
].map(
    {
        1: "Married",
        2: "Married",
        3: "Widowed",
        4: "Divorced",
        5: "Separated",
        6: "Never married",
    }
)

# pesex = Sex
# 1 Male
# 2 Female
currentPopulationSurvey.rename({"pesex": "Sex"}, inplace=True, axis=1)

currentPopulationSurvey["Sex"] = currentPopulationSurvey["Sex"].map(
    {1: "Male", 2: "Female"}
)

The CPS Files contains information about all the household family members, but we are **interested only in the respondent**, thus for each unique id, consider only the first row, since that is information about the respondent.

In [26]:
# interested only in respondent's data
# respondent is the first entry per unique id
currentPopulationSurvey.drop_duplicates(subset="unique id", keep="first", inplace=True)

First few rows, after processing the currentPopulationSurvey data is as follows:

In [27]:
currentPopulationSurvey.head()

,unique id,Yearly Family income category (in dollars),Marriage Status,Sex
0,20030100013007,"Less than 10,000",Never married,Female
2,20030100013013,"Less than 10,000",Never married,Male
3,20030100013014,"50,000 to 99,999",Married,Male
7,20030100013017,"10,000 to 24,999",Widowed,Female
8,20030100013030,"25,000 to 49,999",Married,Female


----------------
**Who file:**

The Who file includes codes that indicate who was present during each activity.

In [28]:
whoData = pd.read_csv(
    "data/atuswho.csv", usecols=["tucaseid", "tuactivity_n", "tuwho_code"]
)

# renaming the column headers
whoData.rename(
    {
        "tucaseid": "unique id",
        "tuactivity_n": "activity serial number", # serial number of activity for a respondent in a day
        "tuwho_code": "Activity Partner",
    },
    axis=1,
    inplace=True,
)

# converting the activity partner code into categorical values
whoData["Activity Partner"] = whoData["Activity Partner"].map(
    {
        18: "Alone",
        19: "Alone",
        20: "Spouse",
        21: "Unmarried partner",
        22: "Own household child",
        23: "Grandchild",
        24: "Parent",
        25: "Brother/sister",
        26: "Other related person",
        27: "Foster child",
        28: "Housemate/roommate",
        29: "Roomer/boarder",
        30: "Other nonrelative",
        40: "Own nonhousehold child < 18",
        51: "Parents (not living in household)",
        52: "Other nonhousehold family members < 18",
        53: "Other nonhousehold family members 18 and older (including parents-in-law)",
        54: "Friends",
        55: "Co-workers/colleagues/clients",
        56: "Neighbors/acquaintances",
        57: "Other nonhousehold children < 18",
        58: "Other nonhousehold adults 18 and older",
        59: "Boss or manager",
        60: "People whom I supervise",
        61: "Co-workers",
        62: "Customers",
    }
)

First few rows, after processing the whoData is as follows:

In [29]:
whoData.head()

,unique id,activity serial number,Activity Partner
0,20030100013280,1,Alone
1,20030100013280,2,NaN
2,20030100013280,3,NaN
3,20030100013280,4,Alone
4,20030100013280,5,Spouse


---------------
### Merging Multiple Files :

We are going to merge multiple files into two final csv files, one contains *respondent information*, other contains information about the *activities performed by respondent*.

#### 1. Respondent Data

Combining Respondent Data from ATUS with CPS data using the unique id.

In [30]:
# respondent + cps data
respondentCleanedData = respondentData.merge(currentPopulationSurvey, on="unique id")

Fetching only the age information about the respondent from Roster Data.

In [31]:
# age data from rosterData
ageData = rosterData.loc[rosterData["Activity Partner"] == "Self", ["unique id", "Age"]]

Combine age data with respondent data, using unique id.

In [32]:
# add age to respondentCleanedData
respondentCleanedData = respondentCleanedData.merge(ageData, on="unique id")

First few rows from final cleaned respondent data is as follows:

In [33]:
respondentCleanedData.head()

,unique id,Generalized industry code,Weightage,Respondent Labour Status,Yearly Family income category (in dollars),Marriage Status,Sex,Age
0,20030100013280,Educational and health services,8155463.0,Employed,"50,000 to 99,999",Married,Male,60
1,20030100013344,Manufacturing,1735323.0,Employed,NaN,Married,Female,41
2,20030100013352,Educational and health services,3830528.0,Employed,"50,000 to 99,999",Married,Female,26
3,20030100013848,NaN,6622023.0,Unemployed,"10,000 to 24,999",Married,Female,36
4,20030100014165,Educational and health services,3068387.0,Employed,"50,000 to 99,999",Married,Male,51


Storing the cleaned respondent dataframe in csv file.

In [34]:
respondentCleanedData.to_csv("data/respondentDataCleaned.csv", index=False)

#### 2. Activity Data

Combining activity data with who data to get information about who was with respondent when activity was being performed. The merge is done using unique id and activity serial number.

In [35]:
# activity + who
activityDataCleaned = activityData.merge(
    whoData, on=["unique id", "activity serial number"]
)

Adding the weightage from respondent data, in the activity dataset.

In [36]:
# add weightage from respondent data
activityDataCleaned = activityDataCleaned.merge(
    respondentData[["unique id", "Weightage"]], on="unique id"
)

First few rows from final cleaned activity data is as follows:

In [37]:
activityDataCleaned.head()

,unique id,activity serial number,minutes of activity,Activity Category,Location of activity,Activity Partner,Weightage
0,20030100013280,1,60,"Sports, Exercise, & Recreation",Outdoors away from home,Alone,8155463.0
1,20030100013280,2,30,Personal Care Activities,NaN,NaN,8155463.0
2,20030100013280,3,600,Personal Care Activities,NaN,NaN,8155463.0
3,20030100013280,4,150,"Socializing, Relaxing, and Leisure",Respondent's home or yard,Alone,8155463.0
4,20030100013280,5,5,Eating and Drinking,Respondent's home or yard,Spouse,8155463.0


Storing the cleaned activity dataframe in csv file.

In [38]:
activityDataCleaned.to_csv("data/activityDataCleaned.csv", index=False)

We now perform analysis, please refer **American Time Use Survey Analysis** Jupyter Notebook for more details.